#### 6. Поиск паттернов и обход деревьев зависимостей

стр 116
В этом ноутбуке есть информация по генерации текста (ответ на ворпос)
Поиск паттернов.Комбинирование паттернов.

В этой главе рассматриваются еще два подхода: использование паттернов (закономерностей) последовательностей слов для классификации и генерации текста, а также выделение из высказывания необходимых элементов информации путем обхода его дерева синтаксических зависимостей. Вы познакомитесь с утилитой Matcher библиотеки spaCy, предназначенной для поиска закономерностей. А также узнаете, в каких случаях для определения нужного подхода к обработке все равно приходится учитывать контекст.

Бвают совершенно различные предложения, но с одинаковыми паттернами (частями речи и зависимости между нимим)

In [102]:
import spacy as sp
nlp = sp.load('ru_core_news_lg')

In [103]:
doc1 = nlp("Я хочу на море")
doc2 = nlp("Мы летим в горы")

In [104]:
for i in range(len(doc1)):
    print(doc1[i].text, doc2[i].text, doc1[i].dep_,doc2[i].dep_)

Я Мы nsubj nsubj
хочу летим ROOT ROOT
на в case case
море горы obl obl


Видно, что предложения совсем рахные, но паттерн у них одинкаовый: nsubj+ROOT+case+obl

подробней об dep_ можно прочитать в start.ipynb // если коротко, то это метка синтакчической зависимости. Например, ROOT -означает, что это слово-смысловой глагол

поиск паттернов лучше. Потому что могут быть различные предложения по содержанию и смыслу, но одинаковые по построению

In [105]:
# ПОПРОБУЙТЕ САМИ стр 118
doc = nlp("Я хочу на море.Мы летим в горы.")
for item in doc.sents:
    sentence = list(item)
    for word in sentence:
        print(word.text,word.dep_)
    print(5*"--")

# эффективней создавать один контейнер doc и приминять один раз nlp. Чем создавать два контейнера и проходить по ним. 
# Используй doc.sents 

Я nsubj
хочу ROOT
на case
море obl
. punct
----------
Мы nsubj
летим ROOT
в case
горы obl
. punct
----------


#### Проверка высказывания на соответствие паттерну

На практике же обычно нет смысла сравнивать предложения друг с другом для выяснения, объединяет ли их один паттерн. Вместо этого стоит проверить полученное предложение на соответствие нужному нам паттерну.

In [106]:
# попробуем увидеть, как паттерн можно отследить 
# nsubj+ROOT+case+obl
doc = nlp("Я хочу на море.")

for item in doc:
    print(5*'--')
    print('TEXT', item.text, item.dep_)
    print('HEAD', item.head, item.dep_)
    print("<children")
    for g in item.children:
        print(g, g.dep_)
    print('/children>')
    print(5*'--')

----------
TEXT Я nsubj
HEAD хочу nsubj
<children
/children>
----------
----------
TEXT хочу ROOT
HEAD хочу ROOT
<children
Я nsubj
море obl
. punct
/children>
----------
----------
TEXT на case
HEAD море case
<children
/children>
----------
----------
TEXT море obl
HEAD хочу obl
<children
на case
/children>
----------
----------
TEXT . punct
HEAD хочу punct
<children
/children>
----------


In [107]:
def dep_pattern(doc):
    # если паттерн есть в предложении, то вернем true 
    for i in range(len(doc)-1):
        if doc[i].dep_ == "nsubj":
            #print(doc[i])
            if doc[i].head.dep_ == "ROOT":
            #    print(doc[i].head)
                for child in doc[i].head.children:
                    if child.dep_ =="obl":
                        return True

In [108]:
doc1 = nlp("Я хочу на море")
doc2 = nlp("Мы летим в горы")
doc3 = nlp("Я купил машину")
doc4 = nlp("Мы летим в горы, а потом на море")

testing = [doc1, doc2, doc3, doc4]


for test in testing:
    print(test.text)
    if dep_pattern(test):
        print("found")
    else:
        print("dint found")

Я хочу на море
found
Мы летим в горы
found
Я купил машину
dint found
Мы летим в горы, а потом на море
found


#### Использование утилиты Matcher библиотеки spaCy для поиска паттернов последовательностей слов

![alt text](matcher.jpeg "matcher")

In [109]:
from spacy.matcher import Matcher

In [110]:
# DEP обращаемся по токену 
# nsubj+ROOT+case+obl

matcher = Matcher(nlp.vocab)
pattern = [{"DEP":"nsubj"},{"DEP":"ROOT"},{"DEP":"case"},{"DEP":"obl"}]
matcher.add("NsubjRootcaseobl", [pattern])

doc = nlp("Я хочу на море. а он не хочет.")

match = matcher(doc)
for match_id, start, end in match:
    span = doc[start:end]
    print("SPAN:", span.text)
    print('Позиция в доке: ',start,'-',end)



SPAN: Я хочу на море
Позиция в доке:  0 - 4


Матчер позволяет искать паттерны только, когда слова идут друг за другом. Если паттерн у нас такой: nsubj+ROOT+..<какой то текст>..+case+obl ; то матчер у нас не сможет работать. Далее посмотрим, как ещё можно искать паттерны в тексте.

#### Применение нескольких паттернов

Одно высказывание можно сопоставить с несколькими паттернами, дабы убедиться, что оно удовлетворяет всем нужным условиям. Например, с двумя паттернами: с реализующим последовательность меток зависимости (как обсуждалось в подразделе «Проверка высказывания на соответствие паттерну» на с. 119) и с проверяющим на соответствие последовательности тегов частей речи. Это может пригодиться, скажем, если будет нужно убедиться, что в роли прямого дополнения в высказывании выступает личное местоимение. В таком случае можно начать поиск соответствующего этому местоимению существительного, упомянутого где-то в другом месте текста.

Напоминим - 
dep - это то, чем является слово в предложении по отношению к другим словам.

pos - это часть речи. (PRON, VERB)

In [111]:
#увидим, что dep паттерн может совпасть, а pos паттерн совсем другой. Тогда нам нужна дополнительная функция 
#которая бы отбирала нужные части речи.

doc = nlp("Я хочу на море.Мы летим в горы.")
for item in doc.sents:
    sentence = list(item)
    for word in sentence:
        print(word.text,word.dep_, word.tag_)
    print(5*"--")



Я nsubj PRON
хочу ROOT VERB
на case ADP
море obl NOUN
. punct PUNCT
----------
Мы nsubj PRON
летим ROOT VERB
в case ADP
горы obl NOUN
. punct PUNCT
----------


In [112]:

doc = nlp("Мы хотим на марс.")

# ИЩЕМ: nsubj+ROOT+case+obl
#       PRON +VERB+ --+---

# уже было
def dep_pattern(doc):
    # если паттерн есть в предложении, то вернем true 
    for i in range(len(doc)-1):
        if doc[i].dep_ == "nsubj":
            #print(doc[i])
            if doc[i].head.dep_ == "ROOT":
            #    print(doc[i].head)
                for child in doc[i].head.children:
                    if child.dep_ =="obl":
                        return True

#дополняем \\ Можно точечно указать части речи, которые нас интересуют  
def pos_pattern(doc):
    for token in doc:
        if token.dep_ == 'nsubj' and token.tag_ != 'PRON':
            return False
        if token.dep_ == 'ROOT' and token.tag_ != 'VERB':
            return False
    return True
    
# тестируем
if dep_pattern(doc) and pos_pattern(doc):
    print('Found')
else:
    print('Not found')

Found


#### Создание паттернов на основе пользовательских признаков

При создании паттерна последовательности слов может возникнуть необходимость расширить функциональность лингвистических признаков, предоставляемых spaCy, подогнав их под свои задачи. Например, может понадобиться, чтобы предыдущий сценарий распознавал паттерн, в котором учитывается число местоимений (единственное или множественное). Это полезно в том случае, если нужно найти существительное, к которому относится местоимение.

Допустим нужно изучить существительное. Тогда нам нужно бы найти местоимение во множественном числе и найти относящееся к нему существительное.

следующем сценарии описывается функция pron_pattern, которая находит все прямые дополнения в передаваемом ей предложении, определяет, является ли прямое дополнение личным местоимением, а затем выясняет, в каком оно числе: единственном или множественном. Далее функция применяется к примеру предложения после проверки его на соответствие двум паттернам, описанным в подразделах «Проверка высказывания на соответствие паттерну» на с. 119 и «Применение нескольких паттернов» на с. 122.

In [113]:
# исспользуем наши dep_pattern и pos_pattern

def pron_pattern(doc):
    plural = ['Мы','нам','они','им']
    for token in doc:
        if token.dep_ == 'nsubj' and token.tag_ == 'PRON':   # обнаруживаем местоимение в паттерне играющее роль nsubj
            if token.text in plural:                        # если это местоимение в множественном числе 
                return 'plural'
            else:
                return 'singular'
    return 'not found'



print(30*"--")
doc = nlp(u'Мы летим в горы с кошками')
print(doc.text)
if dep_pattern(doc) and pos_pattern(doc):
    print('Found:', 'the pronoun in position of direct object is',
    pron_pattern(doc))
else:
    print('Not found')

print(30*"--")
doc = nlp(u'Я хочу марс.')
print(doc.text)
if dep_pattern(doc) and pos_pattern(doc):
    print('Found:', 'the pronoun in position of direct object is',
    pron_pattern(doc))
else:
    print('Not found')

print(30*"--")

------------------------------------------------------------
Мы летим в горы с кошками
Found: the pronoun in position of direct object is plural
------------------------------------------------------------
Я хочу марс.
Not found
------------------------------------------------------------


#### Выбор применяемых паттернов

Столь слабое сцепление между паттернами позволяет использовать их совместно или по отдельности. Например, функцию dep_pattern, проверяющую предложение на соответствие паттерну «подлежащее + вспомогательный глагол + глагол + ... + прямое дополнение...», можно использовать вместе с функцией для паттерна «существительное + вспомогательный модальный глагол + глагол в неопределенной форме + ... + существительное...», дабы убедиться, что и подлежащее, и прямое дополнение в предложении — существительные. Этим двум паттернам будет удовлетворять следующий пример предложения

Короче: паттерны можно и нужно комбинировать, чтобы писать меньше кода.

#### Применение паттернов последовательностей слов в чат-ботах для генерации высказываний

С помощью паттернов последовательностей слов можно реагировать на высказывания пользователя и другим образом — например, генерируя подходящие утвердительные высказывания.

Для подобной генерации текста можно воспользоваться паттернами, реализованными в предыдущих разделах. Перечень шагов подобной генерации выглядит примерно так.

модифицируем действия. Сдеалем не так, как в книге, чтобы подходило для русского языка

1. Проверить вводимые в диалоговом режиме данные с помощью описанных выше функций dep_pattern и pos_pattern и найти высказывание, которое удовлетворяет паттернам 
3. Найти обозначаемое этим местоимением существительное путем поиска существительного, которое управляет наше case местоимение


In [120]:
from spacy import displacy # вот так могу, вот так вау
doc = nlp("Мы хотим на марс")
for ent in doc.ents:
    print(ent.text,ent.start_char, ent.end_char, ent.label_)  
displacy.render(doc, style = 'dep')

марс 12 16 LOC


4. Заменить в найденном при выполнении шага 1 предложении местоимение, играющее роль прямого дополнения, существительным, найденным на шаге 3.
5. Добавить в конец сгенерированного высказывания слово too.

In [115]:
#У нас есть такой поиск паттерна:
# ИЩЕМ, nsubj+ROOT+case+obl  <- dep_pattern()
#   C   PRON +VERB+ --+---   <- pos_pattern()
# Число местоимения (множ. единственное.) <- pron_pattern()

In [116]:
# Попробуем научится генерировать такие ответы.

# Мы хотим на марс -> Я тоже хочу на марс
# Я хочу на море -> Я тоже хочу на море
# Мы летим в горы с кошками -> Я тоже лечу в горы с кошками 
# Мы летим в горы, а потом на марс. -> зачем?

In [117]:
# функция для нахождения подлежащего в тексте. к которому будет относится наш предлог

def find_noun(sents):
    """
    sents - предложения;
    Будем использовать эту функцию для находление нужного подлежащего в предложении
    в котором мы нашли нужный маттерн.
    """
    for sent in reversed(sents):
        for token in sents:
            if token.dep_ == 'case':
                return token.head

    return 'Noun not found'

# Мы хотим на марс.
find_noun(doc) # вау. всё правильно!

марс

In [118]:
# Теперь генерируем наше ответное высказывание, которое соответсвует нашему паттерну.
def gen_answer(doc, noun):
    sent = ''
    for i,token in enumerate(doc):
        if token.dep_ == 'ROOT':
            sent = "Я тоже " + doc[i+1:].text
            return sent

gen_answer(doc,"s")

'Я тоже на марс'

In [119]:
doc = nlp("Я хочу на марс купить там вкусных булочек. А потом послушать красивую музыку и улететь домой в Питербург")
for ent in doc.ents:
    print(ent.text,ent.start_char, ent.end_char, ent.label_)  
displacy.render(doc, style = 'ent')

марс 10 14 LOC
Питербург 95 104 LOC
